In [ ]:
# Google Colab
from google.colab import drive
drive.mount("/content/gdrive")

: 

## Import Library

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import os
import matplotlib.image as mpimg
import glob
import random

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image as im
from keras.utils import load_img, img_to_array
from google.colab import files
from PIL import Image
from tensorflow.keras.layers import Input , Dense , Flatten, MaxPooling2D , Conv2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img

## Load Dataset

In [ ]:
# # importing the zipfile module
# from zipfile import ZipFile

# # loading the archive.zip
# with ZipFile("/content/gdrive/MyDrive/JOKIAN /Gender-Classification/archive.zip", 'r') as zObject:

# # Extracting all untuk mengekstrak folder tsb
# # lokasi yang dipilih untuk ekstrak.
# 	zObject.extractall(
# 		path="/content/gdrive/MyDrive/JOKIAN /Gender-Classification/dataset")

In [ ]:
# Mendefinisikan direktori utama dataset
train_dir = train_dir = '/content/gdrive/MyDrive/JOKIAN /Gender-Classification/dataset/Training/'

## Data Preprocessing

In [ ]:
# Melihat kelas label pada folder train
print(os.listdir(train_dir))

Pada folder train terdapat 4 kelas image yang akan diklasifikasikan

In [ ]:
# Mengetahui banyak data train tiap kelas  
names = []
nums = []
data = {'Label':[],'Jumlah Sampel':[]}

for i in os.listdir(train_dir):
    nums.append(len(os.listdir(train_dir+i)))
    names.append(i)

data['Label']+=names
data['Jumlah Sampel']+=nums

df = pd.DataFrame(data)
df

## Visualization

In [ ]:
# Visualisasi jumlah gambar tiap kelas

plt.figure(figsize=(20,8))
sns.barplot(x=df['Label'],y=df['Jumlah Sampel'])

In [ ]:
# Menampilkan sampel gambar tiap kelas


img_each_class = 1
img_samples = {}
classes = list(names)


for c in classes:
    temp = os.listdir(os.path.join(train_dir, c))[:img_each_class]
    for item in temp:
        img_path = os.path.join(train_dir, c, item)
        img_samples[c] = img_path

for i in img_samples:
    fig = plt.gcf()
    img = mpimg.imread(img_samples[i])
    plt.title(i)
    plt.imshow(img)
    plt.show()

Diketahui bahwa masing-masing gambar sampel dari tiap kelas hewan memiliki ukuran yang berbeda-beda

## Data Preparation




Setelah memahami data, selanjutnya adalah mempersiapkan data sebelum nantinya masuk ke modelling. Penyiapan ini termasuk didalamnya adalah pembagian data (split) menjadi data latih dan validasi. Pembagian data ini diperlukan sebelum nantinya digunakan untuk melatih model yang dibuat serta menghitung akurasi modelnya.

In [ ]:
# Menggunakan ImageDataGenerator untuk preprocessing


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
          validation_split=0.3)

In [ ]:
# Menyiapkan data train dan data validation
train_data = datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    subset='training',
    target_size=(224,224),
    batch_size=32,
    seed=999
)

valid_data = datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    subset='validation',
    target_size=(224,224),
    batch_size=64,
    seed=999
)

Selain membagi data, akan diterapkan juga image augmentation. Hal ini diterapkan berdasarkan data gambar yang telah ditampilkan sebelumnya. Image augmentation yang dilakukan di sini menggunakan layer RandomFlip, RandomRotation, RandomZoom serta Rescaling pada gambar.

In [ ]:
# Image Augmentation
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",input_shape=(224,224,3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.Rescaling(1./255)
  ]
)

Data Augmentation adalah sebuah teknik memanipulasi sebuah data tanpa kehilangan inti atau esensi dari data tersebut. Untuk data berupa Image, kita bisa lakukan rotate, flip, zoom, rescaling.

## Modeling

In [ ]:
path = 'D:\KULIAH\Semester 5\bigprojek\BIG\model\gender_model.h5'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)

**Train Model Transfer Learning MobileNet**

Transfer Learning merupakan teknik yang memanfaatkan model yang sudah ditraining sebelumnya (pretrained model) untuk digunakan mengklasifikasikan dataset yang baru sehingga tidak perlu untuk melakukan training data dari awal (Tan et al. 2018), dan dilakukan penyesuaian pada bagian akhir dari model.

In [ ]:
#Training Model

mobilenet = tf.keras.applications.mobilenet.MobileNet(input_shape=(224 , 224, 3),
                                           include_top=False,
                                           weights='imagenet')

model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(2, activation="softmax" , name="classification"))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0005,momentum=0.9), 
            loss='categorical_crossentropy', 
            metrics = ['accuracy'])

model.summary()

In [ ]:
# load model ckpt
model.load_weights(path)

In [ ]:
# history = model.fit(train_data , validation_data = valid_data , epochs = 3, callbacks=[model_checkpoint_callback])

**Evaluate Model**

In [ ]:
#evaluasi model menggunakan data validasi
model.evaluate(valid_data)

In [ ]:
model.evaluate(train_data)

In [ ]:
# Simpan model dalam bentuk format file .h5 atau .pkl (pickle)
model.save('gender_model.h5')

print('Model Saved Successfully!')

## Visualisasi Training dan Validasi

In [ ]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [ ]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

## Uji Coba Model

In [ ]:
# Menampilkan daftar kelas atau label gambar 
train_data.class_indices

Upload Image

In [ ]:
%matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = load_img(path, target_size=(224,224))
  imgplot = plt.imshow(img)
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=50)
  classes = np.argmax(classes)
  
  print(fn)
  if classes==0:
    print('female')
  elif classes==1:
    print('male')
  else:
    print('Tidak menemukan gender')